In [1]:
import random
import pandas as pd
from glob import glob
import numpy as np  
import os

In [10]:
def train_validate_test_split(img_dir, class_label=0, train_percent=.4, validate_percent=.1, base_train_percent=.4, seed=87264):
    img_list = glob(img_dir + '*.dcm')
    img_list = [os.path.join(s.split('/')[-2], s.split('/')[-1]) for s in img_list] #this is really ugly!
    df = pd.DataFrame({'image':img_list, 'label':[class_label]*len(img_list), 'split':['-']*len(img_list)})
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    base_train_end = int(base_train_percent * m) + train_end
    validate_end = int(validate_percent * m) + base_train_end
    df.loc[perm[:train_end], 'split'] = 'train'
    df.loc[perm[train_end:base_train_end], 'split'] = 'base_train'
    df.loc[perm[base_train_end:validate_end], 'split'] = 'val'
    df.loc[perm[validate_end:], 'split'] = 'test'
    return df

In [11]:
df_hr = train_validate_test_split('/project/catinous/cat_data/hr/', class_label=0)
df_hr_cat = train_validate_test_split('/project/catinous/cat_data/hr_cat/', class_label=1)
df_hr = df_hr.append(df_hr_cat)
df_hr['res'] = 'hr'
df_hr.to_csv('catsinom_hr_dataset.csv')

In [12]:
df_lr = train_validate_test_split('/project/catinous/cat_data/lr/', class_label=0)
df_lr_cat = train_validate_test_split('/project/catinous/cat_data/lr_cat/', class_label=1)
df_lr = df_lr.append(df_lr_cat)
df_lr['res'] = 'lr'
df_lr.to_csv('catsinom_lr_dataset.csv')

In [13]:
df = df_lr.append(df_hr, sort=False)
df.to_csv('catsinom_combined_dataset.csv')

In [14]:
df_hr = train_validate_test_split('/project/catinous/cat_data/hr/', class_label=0)
df_hr_cat_ls = train_validate_test_split('/project/catinous/cat_data/hr_cat_lowshift/', class_label=1)
df_hr = df_hr.append(df_hr_cat_ls)
df_hr['res'] = 'hr'
df = df_lr.append(df_hr, sort=False)
df.to_csv('catsinom_combined_hrlowshift_dataset.csv')

In [29]:
df.sample(100, replace=False).to_csv('test_catsinom_combined_dataset.csv')

In [3]:
df_ds = pd.read_csv('catsinom_combined_dataset.csv', index_col=0)
df_ts = pd.read_csv('catsinom_combined_hrlowshift_dataset.csv', index_col=0)

In [16]:
df.sample(10000, replace=True)

,image,label,split,res
2101,hr/1JDGIABZ_20140819_dicom_202198.dcm,0,train,hr
1319,hr/1GE97R84_20130902_dicom_292609.dcm,0,val,hr
1583,lr/BD6B8048_20070425_dicom_65820.dcm,0,base_train,lr
1971,hr/786B4F46_20130912_dicom_68302.dcm,0,val,hr
1750,hr/1FT2FM85_20140403_dicom_268358.dcm,0,train,hr
...,...,...,...,...
1335,hr_cat_lowshift/6661DB81_20070912_dicom_103059...,1,test,hr
1559,lr/A25ED64F_20131231_dicom_37095.dcm,0,test,lr
413,hr/16001CB5_20080331_dicom_58072.dcm,0,base_train,hr
1790,lr_cat/AC654E04_20110406_dicom_90651.dcm,1,base_train,lr


In [10]:
df_ts.loc[df_ts.res=='hr', 'res'] = 'hr_ts'

In [20]:
split=['base_train', 'train']
selection = np.any([df_ts.split==x for x in split], axis=0)
df_ts_train = df_ts.loc[selection]
df_ts_train = df_ts_train.loc[(df_ts_train.res=='hr_ts') & (df_ts_train.label==1)]

In [21]:
split=['base_train', 'train']
selection = np.any([df_ds.split==x for x in split], axis=0)
df_ds_train = df_ds.loc[selection]
df_ds_train = df_ds_train.loc[(df_ds_train.res=='hr') & (df_ds_train.label==1)]

In [27]:
df_dsts_train_cat = pd.concat((df_ds_train[:500],df_ts_train[500:]))

In [33]:
df_dsts_train_nocat = pd.concat((df_ds_train[:500],df_ts_train[500:]))

In [31]:
split=['base_train', 'train']
selection = np.any([df_ts.split==x for x in split], axis=0)
df_ts_train = df_ts.loc[selection]
df_ts_train = df_ts_train.loc[(df_ts_train.res=='hr_ts') & (df_ts_train.label==0)]

In [28]:
split=['base_train', 'train']
selection = np.any([df_ds.split==x for x in split], axis=0)
df_ds_train = df_ds.loc[selection]
df_ds_train = df_ds_train.loc[(df_ds_train.res=='hr') & (df_ds_train.label==0)]

In [35]:
df_dsts_train = pd.concat((df_dsts_train_nocat, df_dsts_train_cat))

In [40]:
df_dsts_train.groupby(('res','label')).count()

/home/mperkonigg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


image  split
res   label              
hr    0        500    500
      1        500    500
hr_ts 0       1206   1206
      1       1192   1192

In [48]:
df_ts_lr = df_ts.loc[df_ts.res=='lr']

In [44]:
df_dsts_train

,image,label,split,res
0,hr/075DW4AL_20150424_dicom_259874.dcm,0,base_train,hr
2,hr/08CK0MBZ_20140523_dicom_267841.dcm,0,train,hr
3,hr/E2550984_20151007_dicom_83204.dcm,0,base_train,hr
4,hr/1LR6U6BI_20150227_dicom_254482.dcm,0,base_train,hr
6,hr/1KKHW2BK_20140204_dicom_272851.dcm,0,base_train,hr
...,...,...,...,...
2109,hr_cat_lowshift/1JZB1NCL_20140306_dicom_268124...,1,base_train,hr_ts
2110,hr_cat_lowshift/1D683786_20141006_dicom_61052.dcm,1,train,hr_ts
2112,hr_cat_lowshift/E227C188_20120615_dicom_86896.dcm,1,train,hr_ts
2113,hr_cat_lowshift/47048ACE_20080408_dicom_62543.dcm,1,train,hr_ts


In [51]:
df_dsts = pd.concat((df_ts_lr, df_dsts_train))

In [56]:
split=['test', 'val']
selection = np.any([df_ds.split==x for x in split], axis=0)
df_temp = df_ds.loc[selection]
df_temp = df_temp.loc[df_temp.res=='hr']
df_dsts = pd.concat((df_dsts, df_temp))

In [62]:
df_dsts = df_dsts.reset_index()

In [63]:
df_dsts.to_csv('catsinom_combined_dsts_dataset.csv')

In [64]:
df_dsts.groupby('res').count()

,index,image,label,split
res,,,,
hr,1851,1851,1851,1851
hr_ts,3248,3248,3248,3248
lr,3784,3784,3784,3784


In [2]:
df_ds = pd.read_csv('catsinom_combined_dataset.csv', index_col=0)

In [3]:
df_dsts = pd.read_csv('catsinom_combined_dsts_dataset.csv', index_col=0)

In [7]:
df_ds.groupby('split').count()

,image,label,res
split,,,
base_train,3212,3212,3212
test,808,808,808
train,3212,3212,3212
val,801,801,801


In [6]:
df_dsts.groupby('split').count()

,index,image,label,res
split,,,,
base_train,3212,3212,3212,3212
test,1234,1234,1234,1234
train,3212,3212,3212,3212
val,1225,1225,1225,1225


In [8]:
df_dsts.groupby(['split', 'res']).count()

index  image  label
split      res                       
base_train hr       483    483    483
           hr_ts   1216   1216   1216
           lr      1513   1513   1513
test       hr       427    427    427
           hr_ts    426    426    426
           lr       381    381    381
train      hr       517    517    517
           hr_ts   1182   1182   1182
           lr      1513   1513   1513
val        hr       424    424    424
           hr_ts    424    424    424
           lr       377    377    377